In [0]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

carregandos arquivos csv, da base de dados

In [0]:
csv= spark.read.format('csv')\
    .option("header","true")\
    .option("delimiter","\t")\
    .option("encoding", "utf-16")\
    .option("multiline", "true")\
    .load('dbfs:/FileStore/shared_uploads/ivanribeiro_araujo@hotmail.com/*.csv')
	

In [0]:
csv.show(5)

+------------------+--------------------+------+------------+--------+-------------+--------+-------------------+
|        CNPJ emit.|            Emitente|   No.|Data Emissão|Valor NF|Data Registro|Créditos|Situação do Crédito|
+------------------+--------------------+------+------------+--------+-------------+--------+-------------------+
|05.234.750/0001-41|      TX BURGER LTDA| 77882|  01/01/2020|    78,8|   08/01/2020|       0|         A Calcular|
|47.508.411/1476-88|CIA BRASILEIRA DE...| 20087|  01/01/2020|   22,71|   01/01/2020|       0|         A Calcular|
|09.060.964/0108-39|PIMENTA VERDE ALI...| 82366|  01/01/2020|    35,9|   08/01/2020|       0|         A Calcular|
|61.412.110/0307-39|DROGARIA SAO PAUL...|234178|  01/01/2020|   21,77|   08/01/2020|       0|         A Calcular|
|46.952.032/0001-98|PANIFICADORA E CO...|  3305|  01/01/2020|   80,95|   08/01/2020|       0|         A Calcular|
+------------------+--------------------+------+------------+--------+-------------+----

In [0]:
csv.count()

Out[57]: 165464

alterando os nomes das colunas

In [0]:
data = csv.withColumnRenamed('CNPJ emit.','cnp_emit') \
          .withColumnRenamed('Emitente','emitente')\
          .withColumnRenamed('No.','numero') \
          .withColumnRenamed('Data Emissão','data_emissao')\
          .withColumnRenamed('Valor NF','valor_nf')\
          .withColumnRenamed('Data Registro','data_registro')\
          .withColumnRenamed('Créditos','creditos')\
          .withColumnRenamed('Situação do Crédito','situacao_do_credito')

In [0]:
data.show(5)

+------------------+--------------------+------+------------+--------+-------------+--------+-------------------+
|          cnp_emit|            emitente|numero|data_emissao|valor_nf|data_registro|creditos|situacao_do_credito|
+------------------+--------------------+------+------------+--------+-------------+--------+-------------------+
|05.234.750/0001-41|      TX BURGER LTDA| 77882|  01/01/2020|    78,8|   08/01/2020|       0|         A Calcular|
|47.508.411/1476-88|CIA BRASILEIRA DE...| 20087|  01/01/2020|   22,71|   01/01/2020|       0|         A Calcular|
|09.060.964/0108-39|PIMENTA VERDE ALI...| 82366|  01/01/2020|    35,9|   08/01/2020|       0|         A Calcular|
|61.412.110/0307-39|DROGARIA SAO PAUL...|234178|  01/01/2020|   21,77|   08/01/2020|       0|         A Calcular|
|46.952.032/0001-98|PANIFICADORA E CO...|  3305|  01/01/2020|   80,95|   08/01/2020|       0|         A Calcular|
+------------------+--------------------+------+------------+--------+-------------+----

alterandos separador de virgula para ponto, nas celulas (valor_nf e creditos)

In [0]:
data = data.withColumn('valor_nf', regexp_replace('valor_nf', ',', '.'))\
           .withColumn('creditos', regexp_replace('creditos', ',', '.'))

In [0]:
data.show(5)

+------------------+--------------------+------+------------+--------+-------------+--------+-------------------+
|          cnp_emit|            emitente|numero|data_emissao|valor_nf|data_registro|creditos|situacao_do_credito|
+------------------+--------------------+------+------------+--------+-------------+--------+-------------------+
|05.234.750/0001-41|      TX BURGER LTDA| 77882|  01/01/2020|    78.8|   08/01/2020|       0|         A Calcular|
|47.508.411/1476-88|CIA BRASILEIRA DE...| 20087|  01/01/2020|   22.71|   01/01/2020|       0|         A Calcular|
|09.060.964/0108-39|PIMENTA VERDE ALI...| 82366|  01/01/2020|    35.9|   08/01/2020|       0|         A Calcular|
|61.412.110/0307-39|DROGARIA SAO PAUL...|234178|  01/01/2020|   21.77|   08/01/2020|       0|         A Calcular|
|46.952.032/0001-98|PANIFICADORA E CO...|  3305|  01/01/2020|   80.95|   08/01/2020|       0|         A Calcular|
+------------------+--------------------+------+------------+--------+-------------+----

verificando e alterando schema das colunas

In [0]:
data.printSchema()

root
 |-- cnp_emit: string (nullable = true)
 |-- emitente: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- data_emissao: string (nullable = true)
 |-- valor_nf: string (nullable = true)
 |-- data_registro: string (nullable = true)
 |-- creditos: string (nullable = true)
 |-- situacao_do_credito: string (nullable = true)



In [0]:
data = data.withColumn('valor_nf', data.valor_nf.cast('float'))\
        .withColumn('creditos', data.creditos.cast('float'))
        
        

In [0]:
data.printSchema()

root
 |-- cnp_emit: string (nullable = true)
 |-- emitente: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- data_emissao: string (nullable = true)
 |-- valor_nf: float (nullable = true)
 |-- data_registro: string (nullable = true)
 |-- creditos: string (nullable = true)
 |-- situacao_do_credito: string (nullable = true)



In [0]:
data.show()

+------------------+--------------------+------+------------+--------+-------------+--------+-------------------+
|          cnp_emit|            emitente|numero|data_emissao|valor_nf|data_registro|creditos|situacao_do_credito|
+------------------+--------------------+------+------------+--------+-------------+--------+-------------------+
|05.234.750/0001-41|      TX BURGER LTDA| 77882|  01/01/2020|    78.8|   08/01/2020|     0.0|         A Calcular|
|47.508.411/1476-88|CIA BRASILEIRA DE...| 20087|  01/01/2020|   22.71|   01/01/2020|     0.0|         A Calcular|
|09.060.964/0108-39|PIMENTA VERDE ALI...| 82366|  01/01/2020|    35.9|   08/01/2020|     0.0|         A Calcular|
|61.412.110/0307-39|DROGARIA SAO PAUL...|234178|  01/01/2020|   21.77|   08/01/2020|     0.0|         A Calcular|
|46.952.032/0001-98|PANIFICADORA E CO...|  3305|  01/01/2020|   80.95|   08/01/2020|     0.0|         A Calcular|
|42.591.651/2130-58|ARCOS DOURADOS CO...|   438|  01/01/2020|    44.1|   08/01/2020|    

criando uma tabela para fazer consultas com linguagem SQL

In [0]:
data.createTempView("dataset")

fazendo a contagem da situação do credito como 'liberado'

In [0]:
spark.sql("select * from dataset where situacao_do_credito = 'Liberado' ").count()

Out[71]: 160757

select tudo com situação do credtio 'liberado'

In [0]:
spark.sql("select * from dataset where situacao_do_credito = 'Liberado' ").show()

+------------------+--------------------+------+------------+--------+-------------+--------+-------------------+
|          cnp_emit|            emitente|numero|data_emissao|valor_nf|data_registro|creditos|situacao_do_credito|
+------------------+--------------------+------+------------+--------+-------------+--------+-------------------+
|27.282.107/0001-26|SJ COMPANY COMERC...|  6553|  31/12/2019|   32.39|   31/12/2019|     0.0|           Liberado|
|47.508.411/0286-70|CIA BRASILEIRA DE...| 14500|  31/12/2019|     3.6|   31/12/2019|     0.0|           Liberado|
|45.543.915/0375-06|CARREFOUR COMERCI...|226307|  31/12/2019|   49.38|   31/12/2019|     0.0|           Liberado|
|50.974.732/0002-31|COOPERATIVA DE CO...| 22267|  31/12/2019|   79.33|   31/12/2019|    0.17|           Liberado|
|45.543.915/0509-52|CARREFOUR COMERCI...|188662|  31/12/2019|  168.79|   31/12/2019|    0.82|           Liberado|
|13.533.862/0001-21|CHIBRAS SUPERMERC...| 23981|  31/12/2019|   31.26|   31/12/2019|    

selecionando tudo e ordenando pelo maior valor da nota fiscal

In [0]:
spark.sql("select * from dataset  ").orderBy('valor_nf', ascending=False ).show(5)

+------------------+--------------------+------+------------+--------+-------------+--------+-------------------+
|          cnp_emit|            emitente|numero|data_emissao|valor_nf|data_registro|creditos|situacao_do_credito|
+------------------+--------------------+------+------------+--------+-------------+--------+-------------------+
|43.312.503/0001-05|SUPERA FARMA LABO...|169589|  11/05/2020|562500.0|   11/05/2020|     0.0|           Liberado|
|02.471.556/0001-91|Grand Motors Come...|214825|  01/02/2021|128500.0|   01/02/2021|     0.0|           Liberado|
|06.900.979/0002-11|FABERGE DISTRIBUI...| 29314|  07/10/2021| 93000.0|   07/10/2021|     0.0|           Liberado|
|03.678.526/0003-85|ITAVEMA FRANCE VE...|384696|  28/04/2021| 86999.0|   28/04/2021|     0.0|           Liberado|
|03.678.526/0003-85|ITAVEMA FRANCE VE...|384940|  05/05/2021| 86999.0|   05/05/2021|     0.0|           Liberado|
+------------------+--------------------+------+------------+--------+-------------+----

selecionando tudo e ordenando pelo maior valor do credito

In [0]:
spark.sql("select * from dataset  ").orderBy('creditos', ascending=False ).show(5)

+------------------+--------------------+------+------------+--------+-------------+--------+-------------------+
|          cnp_emit|            emitente|numero|data_emissao|valor_nf|data_registro|creditos|situacao_do_credito|
+------------------+--------------------+------+------------+--------+-------------+--------+-------------------+
|78.876.950/0159-50|          CIA HERING| 42139|  26/06/2021|  439.96|   26/06/2021|   290.9|           Liberado|
|52.974.011/0001-76|GRANATA COMERCIO ...|  2142|  23/06/2021|   320.0|   23/06/2021|   290.9|           Liberado|
|60.341.427/0001-85|MERCADINHO KI PRE...|378883|  30/06/2021|  488.25|   30/06/2021|   290.9|           Liberado|
|45.067.147/0019-66|LOJAS ESKALA COME...|100739|  25/06/2021|   320.9|   25/06/2021|   290.9|           Liberado|
|49.669.856/0018-91|RESTOQUE COMERCIO...| 35221|  20/06/2021|   551.7|   20/06/2021|   290.9|           Liberado|
+------------------+--------------------+------+------------+--------+-------------+----